In [27]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from urllib.parse import urljoin
import json

class YayoikenScraper:
    def __init__(self):
        self.base_url = "https://www.yayoiken.com"
        self.menu_list_url = "https://www.yayoiken.com/menu_list/index/13"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.menu_data = []
    
    def get_all_menu_categories(self):
        """メニューページから全カテゴリを取得"""
        try:
            print(f"Fetching menu categories from: {self.menu_list_url}")
            response = self.session.get(self.menu_list_url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            categories = {}
            
            # パターン1: タブメニューを探す
            category_tabs = soup.find_all('a', class_='c-tab-menu__item-inr')
            for tab in category_tabs:
                category_name = tab.text.strip()
                category_url = tab.get('href')
                if category_url:
                    full_url = urljoin(self.base_url, category_url)
                    categories[category_name] = full_url
                    print(f"Found category: {category_name} -> {full_url}")
            
            return categories
            
        except Exception as e:
            print(f"Error fetching categories: {e}")
            import traceback
            traceback.print_exc()
            return {"定食メニュー": self.menu_list_url}
    
    def get_menu_items_from_category(self, category_url):
        """特定のカテゴリからメニューアイテムを取得"""
        try:
            print(f"Fetching menu items from: {category_url}")
            response = self.session.get(category_url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            menu_items = []
            
            # メニューアイテムを取得
            items = soup.find_all('li', class_='c-content-index-menu-list__item')
            
            for item in items:
                try:
                    # リンク要素を取得
                    link = item.find('a', class_='c-content-index-menu-list__item-inr')
                    if not link:
                        continue
                    
                    url = link.get('href')
                    if not url:
                        continue
                    
                    # 商品名を取得
                    title_elem = item.find('p', class_='c-content-index-menu-list__title')
                    product_name = title_elem.text.strip() if title_elem else "N/A"
                    
                    # 価格を取得
                    price_elem = item.find('span', class_='c-content-index-menu-list__price')
                    price = price_elem.text.strip() if price_elem else "N/A"
                    
                    # 価格から「円」、「,」、「"」を削除
                    if price and price != "N/A":
                        price = re.sub(r'[円,"\'"\s　]', '', price)

                    # 画像URLを取得
                    img_elem = item.find('img')
                    picture = img_elem.get('src') if img_elem else "N/A"
                    
                    menu_item = {
                        'product_name': product_name,
                        'price': price,
                        'picture': picture,
                        'url': urljoin(self.base_url, url) if url else "N/A"
                    }
                    
                    menu_items.append(menu_item)
                    print(f"Found: {product_name} - {price}")
                    
                except Exception as e:
                    print(f"Error parsing menu item: {e}")
                    continue
            
            return menu_items
            
        except Exception as e:
            print(f"Error fetching menu items from {category_url}: {e}")
            return []
    
    def get_nutrition_data(self, url):
        """個別商品ページから栄養成分を取得"""
        try:
            print(f"  -> Fetching nutrition data from: {url}")
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            nutrition_data = {
                'calorie': 'N/A',
                'protein': 'N/A',
                'fat': 'N/A',
                'carbohydrate': 'N/A',
                'sugar': 'N/A',
                'fiber': 'N/A',
                'salt': 'N/A'
            }
            
            # パターン1: 新しい構造 - c-table-col9クラス付きのdl要素
            nutrition_table = soup.find('div', class_='c-table__type01')
            if nutrition_table:
                # c-table-col9クラスを持つdl要素を探す
                table_items = nutrition_table.find_all('dl', class_=['c-table__item', 'c-table__item c-table-col9'])
                
                # より広範囲に検索
                if not table_items:
                    table_items = nutrition_table.find_all('dl', class_=re.compile(r'c-table__item'))
                
                print(f"  -> Found {len(table_items)} table items")
                
                for item in table_items:
                    head = item.find('dt', class_='c-table__head')
                    body = item.find('dd', class_='c-table__body')
                    
                    if head and body:
                        head_text = head.get_text(strip=True)
                        body_text = body.get_text(strip=True)
                        
                        print(f"    -> Processing: {head_text} = {body_text}")
                        
                        # 各栄養成分をマッピング（文字の違いも考慮）
                        if '熱量' in head_text:
                            nutrition_data['calorie'] = body_text
                        elif '蛋⽩質' in head_text or 'たんぱく質' in head_text or 'タンパク質' in head_text or '蛋白質' in head_text:
                            nutrition_data['protein'] = body_text
                        elif '脂質' in head_text:
                            nutrition_data['fat'] = body_text
                        elif '炭⽔化物' in head_text or '炭水化物' in head_text:
                            nutrition_data['carbohydrate'] = body_text
                        elif '糖質' in head_text:
                            nutrition_data['sugar'] = body_text
                        elif '食物繊維' in head_text:
                            nutrition_data['fiber'] = body_text
                        elif '⾷塩相当量' in head_text or '食塩相当量' in head_text:
                            nutrition_data['salt'] = body_text
            
            # パターン2: テーブルが見つからない場合、より広範囲に検索
            if nutrition_data['calorie'] == 'N/A':
                print("  -> Trying alternative search methods...")
                
                # すべてのテーブル関連要素を検索
                all_tables = soup.find_all(['table', 'div'], class_=re.compile(r'table'))
                for table in all_tables:
                    table_text = table.get_text()
                    if '熱量' in table_text and '蛋' in table_text:
                        print(f"  -> Found nutrition table: {table.get('class')}")
                        
                        # この要素内からdl要素を検索
                        items = table.find_all('dl')
                        for item in items:
                            head = item.find('dt')
                            body = item.find('dd')
                            if head and body:
                                head_text = head.get_text(strip=True)
                                body_text = body.get_text(strip=True)
                                
                                if '熱量' in head_text:
                                    nutrition_data['calorie'] = body_text
                                elif '蛋⽩質' in head_text or 'たんぱく質' in head_text or 'タンパク質' in head_text or '蛋白質' in head_text:
                                    nutrition_data['protein'] = body_text
                                elif '脂質' in head_text:
                                    nutrition_data['fat'] = body_text
                                elif '炭⽔化物' in head_text or '炭水化物' in head_text:
                                    nutrition_data['carbohydrate'] = body_text
                                elif '糖質' in head_text:
                                    nutrition_data['sugar'] = body_text
                                elif '食物繊維' in head_text:
                                    nutrition_data['fiber'] = body_text
                                elif '⾷塩相当量' in head_text or '食塩相当量' in head_text:
                                    nutrition_data['salt'] = body_text
                        break
            
            # パターン3: 正規表現による直接抽出
            if nutrition_data['calorie'] == 'N/A':
                print("  -> Trying regex extraction...")
                page_text = soup.get_text()
                
                # 各栄養成分を正規表現で抽出
                patterns = {
                    'calorie': r'熱量[^0-9]*?(\d+)',
                    'protein': r'(?:蛋⽩質|たんぱく質|タンパク質|蛋白質)[^0-9]*?([0-9]+\.?[0-9]*)',
                    'fat': r'脂質[^0-9]*?([0-9]+\.?[0-9]*)',
                    'carbohydrate': r'(?:炭⽔化物|炭水化物)[^0-9]*?([0-9]+\.?[0-9]*)',
                    'sugar': r'糖質[^0-9]*?([0-9]+\.?[0-9]*)',
                    'fiber': r'食物繊維[^0-9]*?([0-9]+\.?[0-9]*)',
                    'salt': r'(?:⾷塩相当量|食塩相当量)[^0-9]*?([0-9]+\.?[0-9]*)'
                }
                
                for key, pattern in patterns.items():
                    match = re.search(pattern, page_text)
                    if match:
                        nutrition_data[key] = match.group(1)
                        print(f"    -> Regex found {key}: {match.group(1)}")
            
            print(f"  -> Final nutrition data: {nutrition_data}")
            return nutrition_data
            
        except requests.exceptions.RequestException as e:
            print(f"  -> Request error for {url}: {e}")
            return {
                'calorie': 'Error',
                'protein': 'Error',
                'fat': 'Error',
                'carbohydrate': 'Error',
                'sugar': 'Error',
                'fiber': 'Error',
                'salt': 'Error'
            }
        except Exception as e:
            print(f"  -> Unexpected error for {url}: {e}")
            import traceback
            traceback.print_exc()
            return {
                'calorie': 'Error',
                'protein': 'Error',
                'fat': 'Error',
                'carbohydrate': 'Error',
                'sugar': 'Error',
                'fiber': 'Error',
                'salt': 'Error'
            }
    
    def scrape_all_menus(self):
        """全カテゴリのメニューをスクレイピング"""
        print("Starting comprehensive menu scraping...")
        
        # カテゴリを取得
        categories = self.get_all_menu_categories()
        
        if not categories:
            print("No categories found. Trying to scrape from main page...")
            categories = {"メインメニュー": self.menu_list_url}
        
        all_menu_data = []
        processed_urls = set()  # 重複チェック用
        
        for category_name, category_url in categories.items():
            print(f"\n{'='*60}")
            print(f"Processing Category: {category_name}")
            print(f"{'='*60}")
            
            # カテゴリからメニューアイテムを取得
            menu_items = self.get_menu_items_from_category(category_url)
            
            for i, item in enumerate(menu_items, 1):
                # 重複チェック - URLで判定
                if item['url'] in processed_urls:
                    print(f"  -> Skipping duplicate: {item['product_name']}")
                    continue
                
                processed_urls.add(item['url'])
                
                print(f"\n--- Processing {i}/{len(menu_items)}: {item['product_name']} ---")
                
                # 栄養成分データを取得
                nutrition_data = self.get_nutrition_data(item['url'])
                
                # 指定されたフィールドのみでデータを結合
                complete_data = {
                    'product_name': item['product_name'],
                    'price': item['price'],
                    'picture': item['picture'],
                    'calorie': nutrition_data['calorie'],
                    'protein': nutrition_data['protein'],
                    'fat': nutrition_data['fat'],
                    'carbohydrate': nutrition_data['carbohydrate'],
                    'sugar': nutrition_data['sugar'],
                    'fiber': nutrition_data['fiber'],
                    'salt': nutrition_data['salt'],
                    'url': item['url']
                }
                
                all_menu_data.append(complete_data)
                
                # レート制限対策で少し待機
                time.sleep(0.5)
        
        self.menu_data = all_menu_data
        return all_menu_data
       
    def scrape_from_html(self, html_content):
        """提供されたHTMLからスクレイピングを実行"""
        print("Parsing menu list HTML...")
        menu_items = self.parse_menu_list_html(html_content)
        print(f"Found {len(menu_items)} menu items")
        
        all_data = []
        
        for i, item in enumerate(menu_items, 1):
            print(f"\n--- Processing {i}/{len(menu_items)}: {item['product_name']} ---")
            
            # 栄養成分データを取得
            nutrition_data = self.get_nutrition_data(item['url'])
            
            # 指定されたフィールドのみでデータを結合
            complete_data = {
                'product_name': item['product_name'],
                'price': item['price'],
                'picture': item['picture'],
                'calorie': nutrition_data['calorie'],
                'protein': nutrition_data['protein'],
                'fat': nutrition_data['fat'],
                'carbohydrate': nutrition_data['carbohydrate'],
                'sugar': nutrition_data['sugar'],
                'fiber': nutrition_data['fiber'],
                'salt': nutrition_data['salt'],
                'url': item['url']
            }
            
            all_data.append(complete_data)
            
            # レート制限対策で少し待機
            time.sleep(1)

        self.menu_data = all_data
        return all_data
    
    def parse_menu_list_html(self, html_content):
        """提供されたHTMLから商品URLを抽出"""
        soup = BeautifulSoup(html_content, 'html.parser')
        menu_items = []
        
        # li要素で商品情報を取得
        items = soup.find_all('li', class_='c-content-index-menu-list__item')
        
        for item in items:
            try:
                # リンク要素を取得
                link = item.find('a', class_='c-content-index-menu-list__item-inr')
                if not link:
                    continue
                
                url = link.get('href')
                if not url:
                    continue
                
                # 商品名を取得
                title_elem = item.find('p', class_='c-content-index-menu-list__title')
                product_name = title_elem.text.strip() if title_elem else "N/A"
                
                # 価格を取得
                price_elem = item.find('span', class_='c-content-index-menu-list__price')
                price = price_elem.text.strip() if price_elem else "N/A"
                
                # 価格から「円」、「,」、「"」を削除
                if price and price != "N/A":
                    price = re.sub(r'[円,"\'"\s　]', '', price)
                
                # 画像URLを取得
                img_elem = item.find('img')
                picture = img_elem.get('src') if img_elem else "N/A"
                
                menu_items.append({
                    'product_name': product_name,
                    'price': price,
                    'picture': picture,
                    'url': urljoin(self.base_url, url) if url else "N/A"
                })
                
                print(f"Found: {product_name} - {price}")
                
            except Exception as e:
                print(f"Error parsing menu item: {e}")
                continue
        
        return menu_items
    
    def save_to_csv(self, filename='yayoiken_menu_data.csv'):
        """データをCSVファイルに保存"""
        if not self.menu_data:
            print("No data to save")
            return
        
        fieldnames = ['product_name', 'price', 'picture', 'calorie', 'protein', 'fat', 'carbohydrate', 'sugar', 'fiber', 'salt', 'url']
        
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.menu_data)
        
        print(f"Data saved to {filename}")
    
    def save_to_json(self, filename='yayoiken_menu_data.json'):
        """データをJSONファイルに保存"""
        if not self.menu_data:
            print("No data to save")
            return
        
        with open(filename, 'w', encoding='utf-8') as jsonfile:
            json.dump(self.menu_data, jsonfile, ensure_ascii=False, indent=2)
        
        print(f"Data saved to {filename}")
    
    def display_data(self):
        """取得したデータを表示"""
        if not self.menu_data:
            print("No data to display")
            return
        
        print("\n" + "="*100)
        print("SCRAPED MENU DATA")
        print("="*100)
        
        for i, item in enumerate(self.menu_data, 1):
            print(f"\n【{i}】 {item['product_name']}")
            print(f"価格: {item['price']}")
            print(f"カロリー: {item['calorie']} kcal")
            print(f"タンパク質: {item['protein']} g")
            print(f"脂質: {item['fat']} g")
            print(f"炭水化物: {item['carbohydrate']} g")
            print(f"糖質: {item['sugar']} g")
            print(f"食物繊維: {item['fiber']} g")
            print(f"食塩相当量: {item['salt']} g")
            print(f"画像: {item['picture']}")
            print(f"URL: {item['url']}")
            print("-" * 80)
        
        print(f"\n総メニュー数: {len(self.menu_data)}件")

def main():
    # スクレイパーのインスタンスを作成
    scraper = YayoikenScraper()
    
    print("やよい軒全メニュー情報スクレイピングを開始...")
    print("取得する情報: product_name, price, picture, calorie, protein, fat, carbohydrate, sugar, fiber, salt, url")
    print("価格: 「円」「,」「\"」を削除して数字のみ保存")
    print("重複: URLベースで重複を自動除去")
    
    try:
        # 全カテゴリのメニューデータを取得
        data = scraper.scrape_all_menus()
        
        if not data:
            print("No data was scraped. Please check the website structure.")
            return
        
        # 結果を表示
        scraper.display_data()
        
        # CSVファイルに保存
        scraper.save_to_csv()
        
        # JSONファイルに保存
        scraper.save_to_json()
        
        print(f"\nスクレイピング完了！ {len(data)} 件のメニューデータを取得しました。")
        print("重複データは自動的に除去されました。")
        print("価格データから「円」「,」「\"」が削除されました。")
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

やよい軒全メニュー情報スクレイピングを開始...
取得する情報: product_name, price, picture, calorie, protein, fat, carbohydrate, sugar, fiber, salt, url
価格: 「円」「,」「"」を削除して数字のみ保存
重複: URLベースで重複を自動除去
Starting comprehensive menu scraping...
Fetching menu categories from: https://www.yayoiken.com/menu_list/index/13
No categories found. Trying to scrape from main page...

Processing Category: メインメニュー
Fetching menu items from: https://www.yayoiken.com/menu_list/index/13
Found: さんまの塩焼定食 - 990
Found: 【揚げ出し茄子小鉢付】さんまの塩焼定食 - 1190
Found: 【ミニすき焼き小鉢付】さんまの塩焼定食 - 1310
Found: ～徳島県産阿波尾鶏使用～地鶏まぶし定食 - 1090
Found: ～徳島県産阿波尾鶏使用～【お肉2倍】地鶏まぶし定食 - 1690
Found: 牛バターちゃんぽん焼定食 - 1150
Found: 【お肉２倍】牛バターちゃんぽん焼定食 - 1550
Found: しょうが焼定食 - 860
Found: 肉野菜炒め定食 - 950
Found: チキン南蛮定食 - 990
Found: から揚げ定食 - 890
Found: 特から揚げ定食 - 1080
Found: から揚げ&コロッケ定食 - 890
Found: 味噌かつ煮定食 - 990
Found: コク旨ちゃんぽんとから揚げの定食 - 1100
Found: 大豆ミートのしょうが焼定食 - 830
Found: 大豆ミートの野菜炒め定食 - 920
Found: 大豆ミートのなす味噌と焼魚の定食 - 1120
Found: ブラックアンガスビーフのカットステーキ定食【和風ソース】 - 1430
Found: ブラックアンガスビーフのカットステーキ定食